### 9장 미세 튜닝과 모범 사례

In [2]:
import sys
print(sys.version)

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


### 사전 준비
  * 구글 코랩 환경은 일정 시간이후에 초기화가 되기 때문에 아래 작업을 매번 수행해야 함.
   * my_data.json 파일을 준비한다. (로컬에서 파일 내용을 생성하여 업로드 하거나, 구글 코랩에서 생성한다.)
   * chatgpt.env 파일 생성이 필요.
     * 준비된 chatgpt.env를 내용을 변경하여 업로드 하거나 또는 API_KEY와 ORG_ID를 확인하여 생성한다.
   * pip install openai 설치
     * 설치시 첫 실행시 에러가 발생(23/12) - 해결(다시 한번 실행하면 사라짐)
     ```
     ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
```

### 학습 내용
 * 미세 튜닝 모델 만들기
 * 미세 튜닝 리스트 확인
 * 미세 튜닝 모델 분석 및 삭제하기

mydata.jsonl 파일 내용 - 기본적으로 10줄 이상을 데이터를 필요로 하여 최소 샘플 10줄 준비

```
{"messages": [{"role": "system", "content": "Marv is a factual but humorous chatbot that provides precise and clear explanations in 1-2 sentences."}, {"role": "user", "content": "What's the largest mammal on Earth?"}, {"role": "assistant", "content": "The largest mammal is the blue whale. It's so huge, it might just be the only thing you can't miss in the ocean!"}]}
{"messages": [{"role": "system", "content": "Marv is a factual but humorous chatbot that provides precise and clear explanations in 1-2 sentences."}, {"role": "user", "content": "Name one of Albert Einstein's famous equations."}, {"role": "assistant", "content": "E=mc², Einstein's way of saying mass and energy are pretty chummy."}]}
{"messages": [{"role": "system", "content": "Marv is a factual but humorous chatbot that provides precise and clear explanations in 1-2 sentences."}, {"role": "user", "content": "What's the boiling point of water in Fahrenheit?"}, {"role": "assistant", "content": "Water boils at 212°F, perfect for cooking pasta or a science quiz!"}]}
{"messages": [{"role": "system", "content": "Marv is a factual but humorous chatbot that provides precise and clear explanations in 1-2 sentences."}, {"role": "user", "content": "Who painted the Mona Lisa?"}, {"role": "assistant", "content": "Leonardo da Vinci, the Renaissance man with a flair for mysterious smiles."}]}
{"messages": [{"role": "system", "content": "Marv is a factual but humorous chatbot that provides precise and clear explanations in 1-2 sentences."}, {"role": "user", "content": "What's the largest desert in the world?"}, {"role": "assistant", "content": "The Sahara Desert, famous for its sand, sun, and size!"}]}
{"messages": [{"role": "system", "content": "Marv is a factual but humorous chatbot that provides precise and clear explanations in 1-2 sentences."}, {"role": "user", "content": "Who wrote 'To Kill a Mockingbird'?"}, {"role": "assistant", "content": "Harper Lee, who gave us a novel that's almost as famous as its title."}]}
{"messages": [{"role": "system", "content": "Marv is a factual but humorous chatbot that provides precise and clear explanations in 1-2 sentences."}, {"role": "user", "content": "What's the chemical symbol for gold?"}, {"role": "assistant", "content": "Au, which is also what pirates say when they find it!"}]}
{"messages": [{"role": "system", "content": "Marv is a factual but humorous chatbot that provides precise and clear explanations in 1-2 sentences."}, {"role": "user", "content": "Tell me about the Eiffel Tower."}, {"role": "assistant", "content": "The Eiffel Tower, a small, hardly noticeable landmark in Paris, France."}]}
{"messages": [{"role": "system", "content": "Marv is a factual but humorous chatbot that provides precise and clear explanations in 1-2 sentences."}, {"role": "user", "content": "Who is the first person to set foot on the Moon?"}, {"role": "assistant", "content": "Neil Armstrong, the first moonwalker and space's most famous tourist."}]}
{"messages": [{"role": "system", "content": "Marv is a factual but humorous chatbot that provides precise and clear explanations in 1-2 sentences."}, {"role": "user", "content": "What's the highest mountain in the world?"}, {"role": "assistant", "content": "Mount Everest, the peak that's on everyone's 'highest' bucket list."}]
```

In [4]:
!pip install openai

In [7]:
import os
from openai import OpenAI

def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

init_api()

client = OpenAI(api_key  = os.environ.get("API_KEY"),
                organization  = os.environ.get("ORG_ID"))


### 시간 변경 함수

In [8]:
from datetime import datetime
import pytz

def format_timestamp_to_kst(timestamp):
    """
    타임스탬프가 주어지면 KST (한국 시간대) 시간 문자열로 변환하고, None이면 None을 반환합니다.
    """
    if timestamp is not None:
        utc_time = datetime.utcfromtimestamp(timestamp)
        utc_time = utc_time.replace(tzinfo=pytz.UTC)
        kst_time = utc_time.astimezone(pytz.timezone("Asia/Seoul"))
        return kst_time.strftime('%Y-%m-%d %H:%M:%S')
    return None


print(f"created_at=1701832706 => created_at : {format_timestamp_to_kst(1701832706)}")

created_at=1701832706 => created_at : 2023-12-06 12:18:26


### 01. 기존의 파일 업로드 리스트 확인

In [10]:
# 파일 목록을 가져옵니다.
files = client.files.list()

# 기존에 업로드된 파일 중 동일한 이름의 파일을 찾아 삭제합니다.
for file in files.data:
  change_time = format_timestamp_to_kst(file.created_at)
  print(f"id : {file.id}, created_at : {change_time}, filename : {file.filename}")
  print(file, end="\n\n")

id : file-FVzfAk066UovIhCo3aFyfi08, created_at : 2023-12-06 18:47:31, filename : step_metrics.csv
FileObject(id='file-FVzfAk066UovIhCo3aFyfi08', bytes=28617, created_at=1701856051, filename='step_metrics.csv', object='file', purpose='fine-tune-results', status='processed', status_details=None)

id : file-0KYZqcFqA9c6f6boNGLWh5lQ, created_at : 2023-12-06 18:37:37, filename : drug_malady_data_01_prepared_valid.jsonl
FileObject(id='file-0KYZqcFqA9c6f6boNGLWh5lQ', bytes=32407, created_at=1701855457, filename='drug_malady_data_01_prepared_valid.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

id : file-IPOA67JLwoRVXBRNDOZwFPq1, created_at : 2023-12-06 18:37:36, filename : drug_malady_data_01_prepared_train.jsonl
FileObject(id='file-IPOA67JLwoRVXBRNDOZwFPq1', bytes=129849, created_at=1701855456, filename='drug_malady_data_01_prepared_train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

id : file-boVmKjibUaTDr1cqFAWD8

### 02. OpenAI서버의 미세 조정(fine-tuning)작업 목록 리스트 확인

In [13]:
models = client.fine_tuning.jobs.list()

print("미세 조정 모델 리스트 개수 : ", len(models.data))
print("리스트 상세 내용 객체 출력(전체내용) : ", models.data[0])

for m in models.data:
  created_time = format_timestamp_to_kst(m.created_at)
  finished_time = format_timestamp_to_kst(m.finished_at)

  print(f"id : {m.id}")
  print(f"created_at : {created_time}, finished_at : {finished_time}, status : {m.status}")
  print(f"fine_tuned_model : {m.fine_tuned_model}, model : {m.model},  result_files : {m.result_files}")
  print(f"trained_tokens : {m.trained_tokens}, error : {m.error},  hyperparameters : {m.hyperparameters}")
  print()

미세 조정 모델 리스트 개수 :  8
리스트 상세 내용 객체 출력(전체내용) :  FineTuningJob(id='ftjob-MGf34t0EqAywt8WJypnipJXb', created_at=1701855458, error=None, fine_tuned_model='ft:davinci-002:personal:drug-malady-data:8Sin3adT', finished_at=1701856048, hyperparameters=Hyperparameters(n_epochs=3, batch_size=3, learning_rate_multiplier=2), model='davinci-002', object='fine_tuning.job', organization_id='org-VSSDsvSmnHA1izxVflKKw4HZ', result_files=['file-FVzfAk066UovIhCo3aFyfi08'], status='succeeded', trained_tokens=102660, training_file='file-IPOA67JLwoRVXBRNDOZwFPq1', validation_file='file-0KYZqcFqA9c6f6boNGLWh5lQ')
id : ftjob-MGf34t0EqAywt8WJypnipJXb
created_at : 2023-12-06 18:37:38, finished_at : 2023-12-06 18:47:28, status : succeeded
fine_tuned_model : ft:davinci-002:personal:drug-malady-data:8Sin3adT, model : davinci-002,  result_files : ['file-FVzfAk066UovIhCo3aFyfi08']
trained_tokens : 102660, error : None,  hyperparameters : Hyperparameters(n_epochs=3, batch_size=3, learning_rate_multiplier=2)

id : ftjob-

```
01. client.fine_tuning.jobs.list(): API 클라이언트를 통해 모든 미세 조정 작업의 목록을 가져옵니다.

02. len(models.data): 가져온 목록에서 미세 조정 작업의 총 개수를 계산합니다.

03. 반복문(for m in models.data): 각 미세 조정 작업에 대해 다음 정보를 출력합니다:

id: 미세 조정 작업의 고유 식별자
created_at: 작업이 생성된 날짜 및 시간
status: 작업의 현재 상태 (예: 진행 중, 완료됨)
finished_at: 작업이 완료된 날짜 및 시간
fine_tuned_model: 미세 조정된 모델의 식별자
model: 원본 모델의 식별자
result_files: 작업 결과 파일의 목록
trained_tokens: 훈련에 사용된 토큰의 수
error: 작업 중 발생한 오류 (있는 경우)
hyperparameters: 미세 조정에 사용된 하이퍼파라미터
```

### 03. jsonl파일 리스트 확인 후, 있다면 기존의 리스트 삭제 후, OpenAI 서버에 업로드.

In [23]:
# 파일 목록을 가져옵니다.
files = client.files.list()

# 업로드하려는 파일의 이름
uploading_file_name = "mydata.jsonl"

# 기존에 업로드된 파일 중 동일한 이름의 파일을 찾아 삭제합니다.
for file in files.data:
    if file.filename == uploading_file_name:
        client.files.delete(file.id)
        print(f"Deleted file with ID: {file.id}")

# 새 파일 업로드
response = client.files.create(
  file=open(uploading_file_name, "rb"),
  purpose="fine-tune"
)

print(response)
file_id = response.id
print(file_id)


Deleted file with ID: file-pv1zVZFdVmkPaIh1vwGFO4KD
FileObject(id='file-UFaFoNxdZyEIhK4ACGPd08Vm', bytes=3316, created_at=1702027988, filename='mydata.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
file-UFaFoNxdZyEIhK4ACGPd08Vm


In [24]:
# 파일 목록을 가져옵니다.
files = client.files.list()

# 데이터의 전체 목록 확인
for file in files.data:
  change_time = format_timestamp_to_kst(file.created_at)
  print(f"id : {file.id}, created_at : {change_time}, filename : {file.filename},  bytes : {file.bytes}, purpose : {file.purpose} object : {file.object} ")

id : file-UFaFoNxdZyEIhK4ACGPd08Vm, created_at : 2023-12-08 18:33:08, filename : mydata.jsonl,  bytes : 3316, purpose : fine-tune object : file 
id : file-FVzfAk066UovIhCo3aFyfi08, created_at : 2023-12-06 18:47:31, filename : step_metrics.csv,  bytes : 28617, purpose : fine-tune-results object : file 
id : file-0KYZqcFqA9c6f6boNGLWh5lQ, created_at : 2023-12-06 18:37:37, filename : drug_malady_data_01_prepared_valid.jsonl,  bytes : 32407, purpose : fine-tune object : file 
id : file-IPOA67JLwoRVXBRNDOZwFPq1, created_at : 2023-12-06 18:37:36, filename : drug_malady_data_01_prepared_train.jsonl,  bytes : 129849, purpose : fine-tune object : file 
id : file-boVmKjibUaTDr1cqFAWD8ZMe, created_at : 2023-12-06 18:37:26, filename : drug_malady_data_01_prepared_valid.jsonl,  bytes : 32407, purpose : fine-tune object : file 
id : file-DlAsULtJLe3ewgqVWjlKeG2x, created_at : 2023-12-06 18:37:26, filename : drug_malady_data_01_prepared_train.jsonl,  bytes : 129849, purpose : fine-tune object : file 

### 04. OpenAI의 GPT API를 활용한 미세 조정(fine-tuning) 작업을 수행

* 미세 조정이란 기존의 모델을 특정 데이터셋에 맞게 조정하여 모델의 성능을 특정 작업이나 도메인에 더 적합하게 만드는 과정.

In [25]:
data_id = "file-UFaFoNxdZyEIhK4ACGPd08Vm"

fine_tune_job = client.fine_tuning.jobs.create(
  training_file=data_id,
  model="gpt-3.5-turbo"
)

print(fine_tune_job)
print(fine_tune_job.id)


FineTuningJob(id='ftjob-7ZZdOZAgFhZmFb4qspk2XoWv', created_at=1702028025, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-VSSDsvSmnHA1izxVflKKw4HZ', result_files=[], status='validating_files', trained_tokens=None, training_file='file-UFaFoNxdZyEIhK4ACGPd08Vm', validation_file=None)
ftjob-7ZZdOZAgFhZmFb4qspk2XoWv


```
client.fine_tuning.jobs.create() : 이 메소드는 새로운 미세 조정 작업을 생성하는 데 사용
    training_file="file-MlCjaveJ2sD4rY2Ilbjk1MNn"  :  여기서 training_file 매개변수는 미세 조정에 사용될 트레이닝 데이터 파일의 고유 식별자(ID)를 나타냅니다. 이 식별자는 이전에 API를 통해 업로드된 특정 파일을 가리킵니다.

    model="gpt-3.5-turbo <= 이 매개변수는 사용할 기본 모델을 지정합니다. 여기서는 gpt-3.5-turbo 모델을 사용하도록 설정되어 있습니다. 이 모델은 GPT-3.5 시리즈의 일부로, 대규모 언어 생성 작업에 최적화된 모델입니다.
```

* 추가 설명
  * 이 작업을 통해, 사용자는 기존 GPT-3.5-turbo 모델을 자신의 데이터셋으로 미세 조정하여, 특정 작업이나 도메인에 더 특화된 언어 모델을 생성할 수 있습니다. 예를 들어, 특정 산업 분야의 전문 용어, 스타일 또는 특정 형식의 대화에 더 적합하도록 모델을 조정할 수 있습니다.

In [33]:
models = client.fine_tuning.jobs.list()

print("미세 조정 모델 리스트 개수 : ", len(models.data))

for m in models.data:
  created_time = format_timestamp_to_kst(m.created_at)
  finished_time = format_timestamp_to_kst(m.finished_at)

  print(f"id : {m.id}")
  print(f"created_at : {created_time}, finished_at : {finished_time}, status : {m.status}")
  print(f"fine_tuned_model : {m.fine_tuned_model}, model : {m.model},  result_files : {m.result_files}")
  print(f"trained_tokens : {m.trained_tokens}, error : {m.error},  hyperparameters : {m.hyperparameters}")
  print()

미세 조정 모델 리스트 개수 :  9
id : ftjob-7ZZdOZAgFhZmFb4qspk2XoWv
created_at : 2023-12-08 18:33:45, finished_at : 2023-12-08 18:40:33, status : succeeded
fine_tuned_model : ft:gpt-3.5-turbo-0613:personal::8TRdS2aQ, model : gpt-3.5-turbo-0613,  result_files : ['file-BPBHJKjuUW2x2Hr6kDqEbtxA']
trained_tokens : 6110, error : None,  hyperparameters : Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=2)

id : ftjob-MGf34t0EqAywt8WJypnipJXb
created_at : 2023-12-06 18:37:38, finished_at : 2023-12-06 18:47:28, status : succeeded
fine_tuned_model : ft:davinci-002:personal:drug-malady-data:8Sin3adT, model : davinci-002,  result_files : ['file-FVzfAk066UovIhCo3aFyfi08']
trained_tokens : 102660, error : None,  hyperparameters : Hyperparameters(n_epochs=3, batch_size=3, learning_rate_multiplier=2)

id : ftjob-P1dIj6Gdl4Cab9etkUjAK78f
created_at : 2023-12-06 13:33:02, finished_at : 2023-12-06 13:39:17, status : succeeded
fine_tuned_model : ft:gpt-3.5-turbo-0613:personal::8SdyoWac, model : g

### 05. fine-tuning 작업 상태 확인
 *  작업 ID를 이용하여 현재 미세 조정 작업 전반적인 상태 확인

In [34]:
### 현재 미세 조정 상태 확인
job_id = "ftjob-7ZZdOZAgFhZmFb4qspk2XoWv"
job_status = client.fine_tuning.jobs.retrieve(job_id)

created_time = format_timestamp_to_kst(job_status.created_at)
finished_time = format_timestamp_to_kst(job_status.finished_at)

print(f"id : {m.id}")
print(f"created_at : {created_time}, finished_at : {finished_time}, model : {job_status.model} status : {job_status.status}, error : {job_status.error}")
print()
print("객체 상태 전체 정보 : ", job_status)


id : ftjob-GT16FSNy1xP5iAGU2LuCvzd2
created_at : 2023-12-08 18:33:45, finished_at : 2023-12-08 18:40:33, model : gpt-3.5-turbo-0613 status : succeeded, error : None

객체 상태 전체 정보 :  FineTuningJob(id='ftjob-7ZZdOZAgFhZmFb4qspk2XoWv', created_at=1702028025, error=None, fine_tuned_model='ft:gpt-3.5-turbo-0613:personal::8TRdS2aQ', finished_at=1702028433, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-VSSDsvSmnHA1izxVflKKw4HZ', result_files=['file-BPBHJKjuUW2x2Hr6kDqEbtxA'], status='succeeded', trained_tokens=6110, training_file='file-UFaFoNxdZyEIhK4ACGPd08Vm', validation_file=None)


* 미세 조정 작업과 관련된 이벤트들의 목록을 검색합니다.

In [35]:
### 현재 미세 조정 이벤트 목록 확인
job_id = "ftjob-7ZZdOZAgFhZmFb4qspk2XoWv"

job_list_events =client.fine_tuning.jobs.list_events(fine_tuning_job_id= job_id , limit=10)
print(job_list_events)

for e in job_list_events.data:
  print(e)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-8ZKOFWQucD1HNxreJZDqGagG', created_at=1702028437, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-0jsnwtuqG0cJr0P5ojbICtt5', created_at=1702028434, level='info', message='New fine-tuned model created: ft:gpt-3.5-turbo-0613:personal::8TRdS2aQ', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-a1utaHGxfpPMSDoVP6chEy75', created_at=1702028414, level='info', message='Step 91/100: training loss=0.55', object='fine_tuning.job.event', data={'step': 91, 'train_loss': 0.5537483096122742, 'train_mean_token_accuracy': 0.9473684430122375}, type='metrics'), FineTuningJobEvent(id='ftevent-tkJdyJcNHCGdTqOf6YZxx1Ta', created_at=1702028400, level='info', message='Step 81/100: training loss=0.16', object='fine_tuning.job.event', data={'step': 81, 'train_loss': 0.16153459250926971, 'train_mean_t

### 전체 코드
 * 데이터를 준비 - 10행 이상의 데이터
 * 데이터를 올린다.
 * 데이터 목록 확인
 * 미세 조정 작업 수행
 * 미세 조정 작업 리스트 확인

In [ ]:
# 기본 인증
import os
from openai import OpenAI

def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

init_api()

client = OpenAI(api_key  = os.environ.get("API_KEY"),
                organization  = os.environ.get("ORG_ID"))

# 시간 표시
from datetime import datetime
import pytz

def format_timestamp_to_kst(timestamp):
    """
    타임스탬프가 주어지면 KST (한국 시간대) 시간 문자열로 변환하고, None이면 None을 반환합니다.
    """
    if timestamp is not None:
        utc_time = datetime.utcfromtimestamp(timestamp)
        utc_time = utc_time.replace(tzinfo=pytz.UTC)
        kst_time = utc_time.astimezone(pytz.timezone("Asia/Seoul"))
        return kst_time.strftime('%Y-%m-%d %H:%M:%S')
    return None

# 01. 데이터를 올린다.
# 파일 목록을 가져옵니다.
files = client.files.list()

# 업로드하려는 파일의 이름
uploading_file_name = "mydata.jsonl"

# 기존에 업로드된 파일 중 동일한 이름의 파일을 찾아 삭제합니다.
for file in files.data:
    if file.filename == uploading_file_name:
        client.files.delete(file.id)
        print(f"Deleted file with ID: {file.id}")

# 새 파일 업로드
response = client.files.create(
  file=open(uploading_file_name, "rb"),
  purpose="fine-tune"
)

file_id = response.id
print("작업 ID : ", file_id)

# 02. 데이터의 전체 목록 확인
# 파일 목록을 가져옵니다.
files = client.files.list()

# 데이터의 전체 목록 확인
for file in files.data:
  change_time = format_timestamp_to_kst(file.created_at)
  print(f"id : {file.id}, created_at : {change_time}, filename : {file.filename}")

# 03. 미세 조정 작업 수행
fine_tune_job = client.fine_tuning.jobs.create(
  training_file=file_id,
  model="gpt-3.5-turbo"
)

print("미세 조정 작업 ID : ", fine_tune_job.id)

# 04. 미세 조정 작업 리스트 확인
models = client.fine_tuning.jobs.list()

print("현재 OpenAI 서버의 미세 조정 모델 리스트 개수 : ", len(models.data))

for m in models.data:
  created_time = format_timestamp_to_kst(m.created_at)
  finished_time = format_timestamp_to_kst(m.finished_at)

  print(f"id : {m.id}")
  print(f"created_at : {created_time}, finished_at : {finished_time}, status : {m.status}")
  print(f"fine_tuned_model : {m.fine_tuned_model}, model : {m.model},  result_files : {m.result_files}")
  print(f"trained_tokens : {m.trained_tokens}, error : {m.error},  hyperparameters : {m.hyperparameters}")
  print()

Deleted file with ID: file-ysoAOKTpYCPaugvsb0WEyP6Y
작업 ID :  file-G33NIp5kUkVFmoH7sXixT5tW
id : file-G33NIp5kUkVFmoH7sXixT5tW, created_at : 2023-12-06 13:33:01, filename : mydata.jsonl
id : file-t4k95fuxQoG6QEXNpnj9J7oF, created_at : 2023-12-06 13:30:28, filename : step_metrics.csv
id : file-leVpC6pIFdksKToCIceVCH1X, created_at : 2023-12-06 12:18:26, filename : my_data.jsonl
id : file-CZGUpt2U1qeI7wkLgw3YOCBG, created_at : 2023-12-05 12:52:52, filename : step_metrics.csv
id : file-hAktYdSfFAOxftAX57t6e0Aq, created_at : 2023-12-05 12:46:52, filename : my_data_12.jsonl
id : file-G6kRheWmRA4qHdfWk8FYPiMA, created_at : 2023-12-05 12:07:16, filename : dat_prepared.jsonl
id : file-GyxVlHyBuIAqggd5OfnOBtqr, created_at : 2023-12-05 12:07:01, filename : dat_prepared.jsonl
미세 조정 작업 ID :  ftjob-P1dIj6Gdl4Cab9etkUjAK78f
현재 OpenAI 서버의 미세 조정 모델 리스트 개수 :  7
id : ftjob-P1dIj6Gdl4Cab9etkUjAK78f
created_at : 2023-12-06 13:33:02, finished_at : None, status : validating_files
fine_tuned_model : None, mode

### 06. fine-tuned model 사용해보기

In [40]:
response1 = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0613:personal::8TRdS2aQ",
  messages=[
    {"role": "system", "content": "너는 멋지고 사실적이고 유머러스한 챗봇이야."},
    {"role": "user", "content": "한국의 수도는 어디이니?"}
  ]
)

response2 = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0613:personal::8TRdS2aQ",
  messages=[
    {"role": "system", "content": "너는 멋지고 사실적이고 유머러스한 챗봇이야."},
    {"role": "user", "content": "지구상의 가장 큰 포유동물은 무엇이니?"}
  ]
)

print(response1.choices[0].message.content)
print(response2.choices[0].message.content)


서울이 수도라고 해야지, 한국의 중심이니까요. 그래서 말이 많은 도시라고도 하죠!
지구상에서 가장 큰 포유동물은 고래야. 그들은 거대하고 매우 인상적이지. 하지만, 그들은 물론 사실적으로 우리와 대화하지 않아. 그치만 다행히도 나는 여기 있어서 멋진 대화를 할 수 있어.


### REFERENCE
 * https://platform.openai.com/docs/guides/fine-tuning/use-a-fine-tuned-model